In [2]:
import datetime
from datetime import date
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import requests 

In [3]:
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
import time
chrome_options = webdriver.ChromeOptions()

#chrome_options.add_argument("--headless")
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('"C:\chrome_driver\chromedriver.exe"', chrome_options = chrome_options)
time.sleep(2)

C:\Users\82103\AppData\Local\Temp\ipykernel_9268\1025757986.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('"C:\chrome_driver\chromedriver.exe"', chrome_options = chrome_options)
C:\Users\82103\AppData\Local\Temp\ipykernel_9268\1025757986.py:17: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('"C:\chrome_driver\chromedriver.exe"', chrome_options = chrome_options)


In [6]:
driver.implicitly_wait(10)

url = "https://dart.fss.or.kr/dsab007/main.do"
driver.get(url)
driver.implicitly_wait(10)
selectpath = Select(driver.find_element(By.XPATH, "//*[@id='option']"))
selectpath.select_by_index(2)
input = driver.find_element(By.XPATH, '//*[@id="reportName"]')
input.send_keys('신규시설투자등')
driver.find_element(By.XPATH, '//*[@id="date5"]').send_keys(Keys.ENTER)
driver.find_element(By.XPATH, '//*[@id="date5"]').send_keys(Keys.ENTER)
selectpath = Select(driver.find_element(By.XPATH, '//*[@id="maxResultsCb"]'))
selectpath.select_by_index(3)
input.send_keys(Keys.RETURN)
driver.implicitly_wait(10)
time.sleep(2)
#검색도착
conamelist = []
hreflist = []
for k in range(5):        
    #try:    
        for i in range(1,101):
            #try:
                time.sleep(0.5)
                tds = driver.find_element(By.XPATH, f'//*[@id="tbody"]/tr[{i}]/td[3]')
                a = tds.find_element(By.TAG_NAME, 'a')
                href = a.get_attribute('href')
                coname = driver.find_element(By.XPATH, f'//*[@id="tbody"]/tr[{i}]/td[4]').get_attribute('innerText')
                hreflist.append(href)
                conamelist.append(coname)
                print(i)
          #  except:
             #   print('오류','-'*50)
        driver.find_element(By.XPATH, f'//*[@id="psWrap"]/div[2]/ul/li[{k+2}]/a').send_keys(Keys.ENTER)
        time.sleep(2)
   # except:
     #   break


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
4

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="tbody"]/tr[74]/td[3]"}
  (Session info: chrome=109.0.5414.120)
Stacktrace:
Backtrace:
	(No symbol) [0x00916643]
	(No symbol) [0x008ABE21]
	(No symbol) [0x007ADA9D]
	(No symbol) [0x007E1342]
	(No symbol) [0x007E147B]
	(No symbol) [0x00818DC2]
	(No symbol) [0x007FFDC4]
	(No symbol) [0x00816B09]
	(No symbol) [0x007FFB76]
	(No symbol) [0x007D49C1]
	(No symbol) [0x007D5E5D]
	GetHandleVerifier [0x00B8A142+2497106]
	GetHandleVerifier [0x00BB85D3+2686691]
	GetHandleVerifier [0x00BBBB9C+2700460]
	GetHandleVerifier [0x009C3B10+635936]
	(No symbol) [0x008B4A1F]
	(No symbol) [0x008BA418]
	(No symbol) [0x008BA505]
	(No symbol) [0x008C508B]
	BaseThreadInitThunk [0x765E6BD9+25]
	RtlGetFullPathName_UEx [0x77598FD2+1218]
	RtlGetFullPathName_UEx [0x77598F9D+1165]


In [11]:
tds = driver.find_element(By.XPATH, '//*[@id="tbody"]/tr[23]/td[3]')
a = tds.find_element(By.TAG_NAME, 'a')
href = a.get_attribute('href')
href

'https://dart.fss.or.kr/dsaf001/main.do?rcpNo=20220804900112'

In [38]:
href

'https://dart.fss.or.kr/dsaf001/main.do?rcpNo=20221101800328'

In [39]:
conamelist
invlist = []
mymoneylist = []
mymoneyratlist = []
purposelist = []
for link,coname in zip(hreflist,conamelist):    
    driver.get(link)
    driver.implicitly_wait(1)
    iframes = driver.find_element(By.TAG_NAME, 'iframe')
    driver.switch_to.frame(iframes)
    try: 
        inv = driver.find_element(By.XPATH, '//*[@id="XFormD1_Form0_Table0"]/tbody/tr[2]/td[3]/span').get_attribute('innerText')
        invlist.append(inv.replace(',',''))
        mymoney = driver.find_element(By.XPATH, '//*[@id="XFormD1_Form0_Table0"]/tbody/tr[3]/td[2]/span').get_attribute('innerText')
        mymoneylist.append(mymoney.replace(',', ''))
        mymoneyrat = driver.find_element(By.XPATH, '//*[@id="XFormD1_Form0_Table0"]/tbody/tr[4]/td[2]/span').get_attribute('innerText')
        mymoneylist.append(mymoneyrat)
        purpose = driver.find_element(By.XPATH, '//*[@id="XFormD1_Form0_Table0"]/tbody/tr[6]/td[2]/span').get_attribute('innerText')
        purposelist.append(purpose.strip())
    except:
        inv = driver.find_element(By.XPATH, '//*[@id="XFormD1_Form0_Table0"]/tbody/tr[3]/td[3]/span').get_attribute('innerText')
        invlist.append(inv.replace(',',''))
        mymoney = driver.find_element(By.XPATH, '//*[@id="XFormD1_Form0_Table0"]/tbody/tr[4]/td[2]/span').get_attribute('innerText')
        mymoneylist.append(mymoney.replace(',', ''))
        mymoneyrat = driver.find_element(By.XPATH, '//*[@id="XFormD1_Form0_Table0"]/tbody/tr[5]/td[2]/span').get_attribute('innerText')
        mymoneylist.append(mymoneyrat)
        purpose = driver.find_element(By.XPATH, '//*[@id="XFormD1_Form0_Table0"]/tbody/tr[7]/td[2]/span').get_attribute('innerText')
        purposelist.append(purpose.strip())
    print(coname)
    print(mymoneyrat)


Exception ignored in: <function Service.__del__ at 0x00000162307172E0>
Traceback (most recent call last):
  File "c:\Users\82103\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\common\service.py", line 183, in __del__
    self.stop()
  File "c:\Users\82103\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\common\service.py", line 149, in stop
    self.send_remote_shutdown_command()
  File "c:\Users\82103\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\common\service.py", line 133, in send_remote_shutdown_command
    if not self.is_connectable():
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\82103\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\common\service.py", line 122, in is_connectable
    return utils.is_connectable(self.port)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\82103\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdri

티앤알바이오팹
20.15
대화제약
48.15
세아메카닉스
53.6
지역난방공사
14.5
대주전자재료
20.38
로스웰
16.2
대웅
45.5
해성산업
108.22
해성디에스
108.22
지란지교시큐리티
27.78
켐트로스
27.29
툴젠
10.12
뿌리깊은나무들
3,027.82
대보마그네틱
20.47
대주전자재료
43.13
한솔케미칼
13.66
시스웍
18.38
삼영화학공업
59.02


In [ ]:
len(conamelist)

136

In [ ]:
iframes = driver.find_element(By.TAG_NAME, 'iframe')


In [ ]:
iframes = driver.find_elements_by_tag_name('iframe')
driver.switch_to.frame(iframes)



AttributeError: 'WebDriver' object has no attribute 'find_elements_by_tag_name'

In [ ]:
driver.switch_to.frame(iframes)
